In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv
import re

In [2]:
URL = "https://www.espn.com/college-football/scoreboard/_/group/80/year/2019/seasontype/2/week/1"

In [3]:
with webdriver.Firefox() as driver:
    driver.get(URL)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source,'html')

In [4]:
events = soup.find('div',
                  attrs = {'id': 'events'})

In [5]:
with open("../data/games.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Week', 'HomeTeam', 'AwayTeam', 'HomeId', 'AwayId', 'HomeScore', 'AwayScore'])

In [6]:
with open("../data/games.csv", "a") as csvfile:
    writer = csv.writer(csvfile)
    
    for scoreboard in events.findAll('article',
                                    attrs = {'class' : re.compile("^scoreboard football")}):
        home_id = scoreboard['data-homeid']
        away_id = scoreboard['data-awayid']

        away_team = (scoreboard
                         .find('tr', attrs = {"class": "away"})
                         .find('span', attrs = {"class" : "sb-team-short"})
                         .text)

        away_score = (scoreboard
                         .find('tr', attrs = {"class": "away"})
                         .find('td', attrs = {"class" : "total"})
                         .span.text)

        home_team = (scoreboard
                         .find('tr', attrs = {"class": "home"})
                         .find('span', attrs = {"class" : "sb-team-short"})
                         .text)

        home_score = (scoreboard
                         .find('tr', attrs = {"class": "home"})
                         .find('td', attrs = {"class" : "total"})
                         .span.text)
        
        writer.writerow([1,home_team, away_team, home_id, away_id, home_score, away_score])
    